# Continuous Control

---

You are welcome to use this coding environment to train your agent for the project.  Follow the instructions below to get started!

### 1. Start the Environment

Run the next code cell to install a few packages.  This line will take a few minutes to run!

In [1]:
!pip -q install ./python
print("done")

done


The environments corresponding to both versions of the environment are already saved in the Workspace and can be accessed at the file paths provided below.  

Please select one of the two options below for loading the environment.

In [2]:
from unityagents import UnityEnvironment
import numpy as np

# select this option to load version 1 (with a single agent) of the environment
#env = UnityEnvironment(file_name='/data/Reacher_One_Linux_NoVis/Reacher_One_Linux_NoVis.x86_64')

# select this option to load version 2 (with 20 agents) of the environment
env = UnityEnvironment(file_name='/data/Reacher_Linux_NoVis/Reacher.x86_64')

INFO:unityagents:
'Academy' started successfully!
Unity Academy name: Academy
        Number of Brains: 1
        Number of External Brains : 1
        Lesson number : 0
        Reset Parameters :
		goal_speed -> 1.0
		goal_size -> 5.0
Unity brain name: ReacherBrain
        Number of Visual Observations (per agent): 0
        Vector Observation space type: continuous
        Vector Observation space size (per agent): 33
        Number of stacked Vector Observation: 1
        Vector Action space type: continuous
        Vector Action space size (per agent): 4
        Vector Action descriptions: , , , 


Environments contain **_brains_** which are responsible for deciding the actions of their associated agents. Here we check for the first brain available, and set it as the default brain we will be controlling from Python.

In [3]:
# get the default brain
brain_name = env.brain_names[0]
brain = env.brains[brain_name]

### 2. Examine the State and Action Spaces

Run the code cell below to print some information about the environment.

In [5]:
# reset the environment
env_info = env.reset(train_mode=True)[brain_name]

# number of agents
num_agents = len(env_info.agents)
print('Number of agents:', num_agents)

# size of each action
action_size = brain.vector_action_space_size
print('Size of each action:', action_size)

# examine the state space 
states = env_info.vector_observations
state_size = states.shape[1]
print('There are {} agents. Each observes a state with length: {}'.format(states.shape[0], state_size))
print('The state for the first agent looks like:', states[0])

Number of agents: 20
Size of each action: 4
There are 20 agents. Each observes a state with length: 33
The state for the first agent looks like: [  0.00000000e+00  -4.00000000e+00   0.00000000e+00   1.00000000e+00
  -0.00000000e+00  -0.00000000e+00  -4.37113883e-08   0.00000000e+00
   0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
   0.00000000e+00   0.00000000e+00  -1.00000000e+01   0.00000000e+00
   1.00000000e+00  -0.00000000e+00  -0.00000000e+00  -4.37113883e-08
   0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
   0.00000000e+00   0.00000000e+00   7.90150833e+00  -1.00000000e+00
   1.25147629e+00   0.00000000e+00   1.00000000e+00   0.00000000e+00
  -5.22214413e-01]


### 4. Solution using a DDPG agent

Now it's your turn to train your own agent to solve the environment!  A few **important notes**:
- When training the environment, set `train_mode=True`, so that the line for resetting the environment looks like the following:
```python
env_info = env.reset(train_mode=True)[brain_name]
```
- To structure your work, you're welcome to work directly in this Jupyter notebook, or you might like to start over with a new file!  You can see the list of files in the workspace by clicking on **_Jupyter_** in the top left corner of the notebook.
- In this coding environment, you will not be able to watch the agents while they are training.  However, **_after training the agents_**, you can download the saved model weights to watch the agents on your own machine! 

In [6]:
from ddpg_agent import Agent, ReplayBuffer
from collections import deque
import time
import torch
import pandas as pd

random_seed = 4
train_mode = True

agent = Agent(state_size=state_size, action_size=action_size, random_seed=random_seed)
print(agent)

def ddpg(n_episodes=500, max_t=1000, goal_score=30):
    mean_scores_window = deque(maxlen=100)
    learning_stats = pd.DataFrame(index=range(1, n_episodes+1), 
                                  columns=['Mean Score','Min Score','Max Score','Moving Avg. (100 eps)'])
    learning_stats.index.name = 'Episode'
    
    for i_episode in range(1, n_episodes+1):
        env_info = env.reset(train_mode=train_mode)[brain_name] 
        states = env_info.vector_observations
        scores = np.zeros(num_agents)
        agent.reset()
        
        start_time = time.time()
        
        for t in range(max_t):
            actions = agent.act(states)
            
            env_info = env.step(actions)[brain_name]
            next_states = env_info.vector_observations
            rewards = env_info.rewards
            dones = env_info.local_done
            
            for state, action, reward, next_state, done in zip(states, actions, rewards, next_states, dones):
                agent.step(state, action, reward, next_state, done)
            
            scores += env_info.rewards
            states = next_states
            
            if np.any(dones):
                break
    
        learning_stats.loc[i_episode,'Mean Score'] = mean_score = np.mean(scores)
        learning_stats.loc[i_episode,'Min Score'] = min_score = np.min(scores)
        learning_stats.loc[i_episode,'Max Score'] = max_score = np.max(scores)
        mean_scores_window.append(mean_score)
        learning_stats.loc[i_episode,'Moving Avg. (100 eps)'] = moving_average_score = np.mean(mean_scores_window)
        
        duration = time.time() - start_time
        
        print('\rEpisode {}\tMoving Avg. Score: {:.2f}\tMean: {:.2f}\tMin: {:.2f}\tMax: {:.2f}\tDur.: {:.2f}'
              .format(i_episode, moving_average_score, mean_score, min_score, max_score, duration))

        if i_episode % 5 == 0:
            torch.save(agent.actor_local.state_dict(), 'checkpoint_actor.pth')
            torch.save(agent.critic_local.state_dict(), 'checkpoint_critic.pth')
            learning_stats.to_csv('learning_stats_'+str(i_episode)+'.csv')
            
        if moving_average_score >= goal_score and i_episode >= 100:
            print('Problem Solved after {} epsisodes! Moving Average score: {:.2f}'.format(i_episode, moving_average_score))
            torch.save(agent.actor_local.state_dict(), 'checkpoint_actor.pth')
            torch.save(agent.critic_local.state_dict(), 'checkpoint_critic.pth')
            break
        
    learning_stats.dropna(inplace=True)
    learning_stats.to_csv('learning_stats.csv')

ddpg()

Episode 1	Moving Avg. Score: 0.41	Mean: 0.41	Min: 0.09	Max: 0.99	Dur.: 34.76
Episode 2	Moving Avg. Score: 0.51	Mean: 0.61	Min: 0.09	Max: 1.54	Dur.: 37.44
Episode 3	Moving Avg. Score: 0.61	Mean: 0.80	Min: 0.07	Max: 1.92	Dur.: 38.53
Episode 4	Moving Avg. Score: 0.64	Mean: 0.73	Min: 0.00	Max: 1.70	Dur.: 38.88
Episode 5	Moving Avg. Score: 0.74	Mean: 1.11	Min: 0.00	Max: 2.47	Dur.: 39.08
Episode 6	Moving Avg. Score: 0.86	Mean: 1.48	Min: 0.14	Max: 3.89	Dur.: 40.02
Episode 7	Moving Avg. Score: 0.95	Mean: 1.50	Min: 0.49	Max: 2.53	Dur.: 41.12
Episode 8	Moving Avg. Score: 1.12	Mean: 2.33	Min: 0.45	Max: 5.74	Dur.: 40.70
Episode 9	Moving Avg. Score: 1.29	Mean: 2.61	Min: 1.24	Max: 5.05	Dur.: 42.57
Episode 10	Moving Avg. Score: 1.50	Mean: 3.37	Min: 1.41	Max: 7.44	Dur.: 42.69
Episode 11	Moving Avg. Score: 1.78	Mean: 4.65	Min: 1.94	Max: 13.23	Dur.: 44.57
Episode 12	Moving Avg. Score: 2.09	Mean: 5.42	Min: 2.27	Max: 9.99	Dur.: 44.95
Episode 13	Moving Avg. Score: 2.43	Mean: 6.52	Min: 2.50	Max: 12.89	Dur.: